# PnL Ratio - follow

Market sentiment is high when everyone is profit on paper, and vice versa. Can we use this as an indicator to trade?

## Comment

If it is a true random time series, this strategy will not outperform the buy-and-forget strategy for both absolute and per unit risk measures. The random generation function is basic on the normal distribution which assumes constant variance and independence. However, this is often not true for real financial data, and therefore may have explained the good performance of this strategy on real data. Also, hand picked symbols may have subject to survival bias, since the trader already know that the symbol has performed quite well in the backtest period. 

## Data

In [141]:
from btbox import *
from pandas import Series
import numpy as np

In [142]:
SYMBOL = 'DUMMY'
DATA_START = '2012-01-01'
START = '2014-01-01'
END = '2022-06-30'
WINDOW = 365*2
INTERVAL = 5

In [143]:
np.random.seed(9999)
dfs = {SYMBOL: make_random_ohlcv(DATA_START, END, mu=0.11, sigma=0.25)}

## Benchmark

In [144]:
class BM_AllInAndForget(Strategy):

    def initial(self, b: Broker):
        b.portfolio.trade_target_weight(SYMBOL, 1)


class BM_KeepAtHalf(Strategy):

    @interval(INTERVAL)
    def step(self, b: Broker):
        b.portfolio.trade_target_weight(SYMBOL, 0.5)

## Indicator

In [145]:
def pnl_ratio(win: Series) -> float:
    pnlr = win.rank(pct=True)
    return pnlr[-1]

## Strategy

In [146]:
def ST_FollowPnL(factor):

    class ST(Strategy):
        name = f'ST_FollowPnl({factor})'

        @interval(INTERVAL)
        def step(self, b: Broker):
            win = b.market.get_close_window(SYMBOL)
            pnlr = pnl_ratio(win)
            weight = pnlr * factor
            b.portfolio.trade_target_weight(SYMBOL, weight)
            self.journal.mark(pnlr, 'pnl-ratio')

    return ST

## Backtest

In [147]:
bt = create_backtest(
    [
        BM_AllInAndForget,
        BM_KeepAtHalf,
        ST_FollowPnL(0.5),
        ST_FollowPnL(1.0),
        ST_FollowPnL(1.5),
    ],
    dfs,
    start=START,
    window=WINDOW,
)

In [148]:
results = bt.run()

## Dashboard

In [149]:
results.dashboard_pretty()

,return,cagr,mu,sigma,mdd,duration,sharpe,calmar
BM_AllInAndForget,172.9%,12.54%,14.93%,24.97%,-52.13%,759 days 00:00,0.598,0.287
BM_KeepAtHalf,76.4%,6.91%,7.46%,12.49%,-29.83%,759 days 00:00,0.598,0.250
ST_FollowPnl(0.5),53.3%,5.15%,5.50%,9.67%,-14.97%,239 days 00:00,0.568,0.367
ST_FollowPnl(1.0),116.7%,9.53%,10.98%,19.33%,-28.15%,239 days 00:00,0.568,0.390
ST_FollowPnl(1.5),182.6%,13.00%,16.44%,29.00%,-39.66%,239 days 00:00,0.567,0.414


## Equity Curve

In [150]:
results.plot(log_y=True)

## Evaluation

In [151]:
results['ST_FollowPnl(1.0)'].journals['pnl-ratio'].ffill.plot_line_under_price(SYMBOL)